# LSTM Stock Predictor Using Closing Prices

In this notebook, you will build and train a custom LSTM RNN that uses a 10 day window of Bitcoin closing prices to predict the 11th day closing price. 

You will need to:

1. Prepare the data for training and testing
2. Build and train a custom LSTM RNN
3. Evaluate the performance of the model

## Data Preparation

In this section, you will need to prepare the training and testing data for the model. The model will use a rolling 10 day window to predict the 11th day closing price.

You will need to:
1. Use the `window_data` function to generate the X and y values for the model.
2. Split the data into 70% training and 30% testing
3. Apply the MinMaxScaler to the X and y values
4. Reshape the X_train and X_test data for the model. Note: The required input format for the LSTM is:

```python
reshape((X_train.shape[0], X_train.shape[1], 1))
```

In [349]:
import numpy as np
import pandas as pd
import hvplot.pandas

In [350]:
# Set the random seed for reproducibility
# Note: This is for the homework solution, but it is good practice to comment this out and run multiple experiments to evaluate your model
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [351]:
# Load the fear and greed sentiment data for Bitcoin
df = pd.read_csv('btc_sentiment.csv', index_col="date", infer_datetime_format=True, parse_dates=True)
df = df.drop(columns="fng_classification")
df.head()

,fng_value
date,
2019-07-29,19
2019-07-28,16
2019-07-27,47
2019-07-26,24
2019-07-25,42


In [352]:
# Load the historical closing prices for bitcoin
df2 = pd.read_csv('btc_historic.csv', index_col="Date", infer_datetime_format=True, parse_dates=True)['Close']
df2 = df2.sort_index()
df2.tail()

Date
2019-07-25    9882.429688
2019-07-26    9847.450195
2019-07-27    9478.320313
2019-07-28    9531.769531
2019-07-29    9529.889648
Name: Close, dtype: float64

In [353]:
# Join the data into a single DataFrame
df = df.join(df2, how="inner")
df.tail()

,fng_value,Close
2019-07-25,42,9882.429688
2019-07-26,24,9847.450195
2019-07-27,47,9478.320313
2019-07-28,16,9531.769531
2019-07-29,19,9529.889648


In [354]:
df.head()

,fng_value,Close
2018-02-01,30,9114.719727
2018-02-02,15,8870.820313
2018-02-03,40,9251.269531
2018-02-04,24,8218.049805
2018-02-05,11,6937.080078


In [355]:
# This function accepts the column number for the features (X) and the target (y)
# It chunks the data up with a rolling window of Xt-n to predict Xt
# It returns a numpy array of X any y
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i:(i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [356]:
# Predict Closing Prices using a 10 day window of previous closing prices
# Try a window size anywhere from 1 to 10 and see how the model performance changes
window_size = 14

# Column index 1 is the `Close` column
feature_column = 1
target_column = 1
X, y = window_data(df, window_size, feature_column, target_column)

In [357]:
# Use 70% of the data for training and the remainder for testing
split = int(0.7 * len(X))
X_train = X[: split - 1]
X_test = X[split:]
y_train = y[: split - 1]
y_test = y[split:]

In [358]:
# Use MinMaxScaler to scale the data between 0 and 1. 
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
scaler.fit(y)
y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)

In [359]:
# Reshape the features for the model
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
print (f"X_train sample values:\n{X_train[:5]} \n")
print (f"X_test sample values:\n{X_test[:5]}")

X_train sample values:
[[[0.60761794]
  [0.58242373]
  [0.62172321]
  [0.51499412]
  [0.38267307]
  [0.46160996]
  [0.4503991 ]
  [0.5193988 ]
  [0.56445096]
  [0.55127638]
  [0.5012101 ]
  [0.58660203]
  [0.5487353 ]
  [0.64593307]]

 [[0.58242373]
  [0.62172321]
  [0.51499412]
  [0.38267307]
  [0.46160996]
  [0.4503991 ]
  [0.5193988 ]
  [0.56445096]
  [0.55127638]
  [0.5012101 ]
  [0.58660203]
  [0.5487353 ]
  [0.64593307]
  [0.70255153]]

 [[0.62172321]
  [0.51499412]
  [0.38267307]
  [0.46160996]
  [0.4503991 ]
  [0.5193988 ]
  [0.56445096]
  [0.55127638]
  [0.5012101 ]
  [0.58660203]
  [0.5487353 ]
  [0.64593307]
  [0.70255153]
  [0.71856064]]

 [[0.51499412]
  [0.38267307]
  [0.46160996]
  [0.4503991 ]
  [0.5193988 ]
  [0.56445096]
  [0.55127638]
  [0.5012101 ]
  [0.58660203]
  [0.5487353 ]
  [0.64593307]
  [0.70255153]
  [0.71856064]
  [0.81240436]]

 [[0.38267307]
  [0.46160996]
  [0.4503991 ]
  [0.5193988 ]
  [0.56445096]
  [0.55127638]
  [0.5012101 ]
  [0.58660203]
  [0.5487

---

## Build and Train the LSTM RNN

In this section, you will design a custom LSTM RNN and fit (train) it using the training data.

You will need to:
1. Define the model architecture
2. Compile the model
3. Fit the model to the training data

### Hints:
You will want to use the same model architecture and random seed for both notebooks. This is necessary to accurately compare the performance of the FNG model vs the closing price model. 

In [360]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [361]:
# Build the LSTM model. 
# The return sequences need to be set to True if you are adding additional LSTM layers, but 
# You don't have to do this for the final layer. 

# Define the LSTM RNN model.
model = Sequential()

number_units =14
dropout_fraction = 0.2

# Layer 1
model.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train.shape[1], 1))
    )
model.add(Dropout(dropout_fraction))

# Layer 2
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))

# Layer 3
model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))

# Output layer
model.add(Dense(1))

In [362]:
# Compile the model
model.compile(optimizer="adam", loss="mean_squared_error")

In [363]:
# Summarize the model
model.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_72 (LSTM)               (None, 14, 14)            896       
_________________________________________________________________
dropout_72 (Dropout)         (None, 14, 14)            0         
_________________________________________________________________
lstm_73 (LSTM)               (None, 14, 14)            1624      
_________________________________________________________________
dropout_73 (Dropout)         (None, 14, 14)            0         
_________________________________________________________________
lstm_74 (LSTM)               (None, 14)                1624      
_________________________________________________________________
dropout_74 (Dropout)         (None, 14)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 1)               

In [364]:
# Train the model
# Use at least 10 epochs
# Do not shuffle the data
# Experiement with the batch size, but a smaller batch size is recommended

model.fit(X_train, y_train, epochs=30, shuffle=False, batch_size=1, verbose=1)

Epoch 1/30
368/368 [==============================] - 4s 12ms/step - loss: 0.0283
Epoch 2/30
368/368 [==============================] - 4s 12ms/step - loss: 0.0218
Epoch 3/30
368/368 [==============================] - 4s 12ms/step - loss: 0.0228
Epoch 4/30
368/368 [==============================] - 5s 13ms/step - loss: 0.0213
Epoch 5/30
368/368 [==============================] - 4s 12ms/step - loss: 0.0177
Epoch 6/30
368/368 [==============================] - 4s 12ms/step - loss: 0.0173
Epoch 7/30
368/368 [==============================] - 5s 12ms/step - loss: 0.0160
Epoch 8/30
368/368 [==============================] - 5s 13ms/step - loss: 0.0131
Epoch 9/30
368/368 [==============================] - 4s 12ms/step - loss: 0.0124
Epoch 10/30
368/368 [==============================] - 5s 13ms/step - loss: 0.0118: 0s - l
Epoch 11/30
368/368 [==============================] - 5s 13ms/step - loss: 0.0101
Epoch 12/30
368/368 [==============================] - 5s 12ms/step - loss: 0.0089
Epoch

---

## Model Performance

In this section, you will evaluate the model using the test data. 

You will need to:
1. Evaluate the model using the `X_test` and `y_test` data.
2. Use the X_test data to make predictions
3. Create a DataFrame of Real (y_test) vs predicted values. 
4. Plot the Real vs predicted values as a line chart

### Hints
Remember to apply the `inverse_transform` function to the predicted and y_test values to recover the actual closing prices.

In [365]:
# Evaluate the model
model.evaluate(X_test, y_test)

5/5 [==============================] - 0s 4ms/step - loss: 0.0147


0.014683479443192482

In [366]:
# Make some predictions
predicted = model.predict(X_test)

In [367]:
# Recover the original prices instead of the scaled version
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

In [368]:
# Create a DataFrame of Real and Predicted values
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
})
stocks.head()

,Real,Predicted
0,3974.050049,3879.670410
1,3937.040039,3915.094971
2,3983.530029,3940.290283
3,4149.089844,3956.350342
4,3771.620117,3977.605713


In [369]:
# Plot the real vs predicted values as a line chart
stocks.hvplot()

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [370]:
df.head()

,fng_value,Close
2018-02-01,30,9114.719727
2018-02-02,15,8870.820313
2018-02-03,40,9251.269531
2018-02-04,24,8218.049805
2018-02-05,11,6937.080078


In [371]:

#add two columns of rolling close and rolling fng, just to check correlations

df['close_rolling'] = df["Close"].rolling(10).mean()
df['fng_rolling'] = df[" fng_value"].rolling(10).mean()
df.dropna(inplace=True)
df.head()

,fng_value,Close,close_rolling,fng_rolling
2018-02-10,54,8569.290039,8321.272022,29.2
2018-02-11,31,8084.609863,8218.261035,29.3
2018-02-12,42,8911.269531,8222.305957,32.0
2018-02-13,35,8544.690430,8151.648047,31.5
2018-02-14,55,9485.639648,8278.407031,34.6


In [372]:
#Check correlations
close = df['Close']
fng_rolling = df["fng_rolling"]


#Find correlation between close and fng_rolling average
correlation_with_fng_rolling = close.corr(fng_rolling)
correlation_with_fng_rolling

0.4218515211913012

In [373]:

#Find correlation between close and close rolling average
close_rolling = df["close_rolling"]

#Find correlation between close and fng_rolling average
correlation_with_close_rolling = close.corr(close_rolling)
correlation_with_close_rolling



0.9737136079799448

In [ ]:
#The close is more closely correlated with the 10 day close moving average than with the 10 day fng moving average.
#Supports the conclusion that the RNN LSTM model using closing prices is better than the model using FNG values.